<a href="https://colab.research.google.com/github/Greta-gerasimov/NLP_1/blob/L10-11%2FMT_with%2Fout_attention/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0__Maching_translate__L10_hw10_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание
Разобраться с моделькой перевода как она устроена запустить для перевода с русского на английский (при желании можно взять другие пары языков) два варианта с вниманием и без внимания оценить качество насколько корректно переводит

In [4]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [5]:
# Загружаем файл
path_to_file = "/content/ita.txt"

In [6]:
#просмотр файла
f = open(path_to_file)
for line in f:
    print(line)
    break

Hi.	Ciao!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #607364 (Cero)



## **Preprocessing**

In [7]:

def preprocess_sentence(w):
  #переводим предложение к нижнему регистру и удалем начальные и конечные пробелы
    w = w.lower().strip()

  # отделяем пробелом слово и следующую за ним пунктуацию
  # пример: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

  # все, кроме букв и знаков пунктуации, заменяем пробелом
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  #удаляем лишние пробелы в начале и конце
    w = w.strip()

  # создаем начало и конец последовательности
  # теперь модель знает, где начинать и заканчивать предсказания
    w = '<start> ' + w + ' <end>'
    return w

In [8]:
preprocess_sentence("The long way is beginning  with one step!")

'<start> the long way is beginning with one step ! <end>'

In [9]:
# 1. Убираем акценты
# 2. Очищаем предложения
# 3. Возвращаем пары слов: [isl, ENG]
def create_dataset(path, num_examples):
  #считываем строки файла
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  #каждую строку разделяем на пробелы, берем первые 2 слова, препроцессим их и возвращаем пару
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [10]:
#пример работы
en, it = create_dataset(path_to_file, None)
print(it[18])
print(en[18])

<start> salta . <end>
<start> jump . <end>


In [11]:
# количество данных в датасете
len(en), len(it)

(374932, 374932)

In [12]:
def tokenize(lang):
      #токенизируем текст, отфильтвовываем пробелы
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
      #обновляем внутренний словарь на основе lang
    lang_tokenizer.fit_on_texts(lang)
      #преобразуем каждый элемент из lang в последовательность чисел
    tensor = lang_tokenizer.texts_to_sequences(lang)
      #преобразуем тензор в матрицу (кол-во тензоров * max-длина),
      #при этом короткие последовательности заполняем нулями сзади, а длинные -- обрезаем сзади
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [13]:
def load_dataset(path, num_examples=None):
      # создаем очищенные анг (выходные), it (входные) пары
    targ_lang, inp_lang = create_dataset(path, num_examples)
    #применяем токенизацию к каждому элементы из пары
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [14]:
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

input_tensor, target_tensor

(array([[   1, 1212,   39, ...,    0,    0,    0],
        [   1, 1212,    3, ...,    0,    0,    0],
        [   1, 3632,   39, ...,    0,    0,    0],
        ...,
        [   1,  433,    8, ...,    0,    0,    0],
        [   1,  433,  127, ...,    0,    0,    0],
        [   1,  433,  554, ...,    0,    0,    0]], dtype=int32),
 array([[   1, 1044,    3, ...,    0,    0,    0],
        [   1, 1044,    3, ...,    0,    0,    0],
        [   1,  354,   48, ...,    0,    0,    0],
        ...,
        [   1,  484,   70, ...,    0,    0,    0],
        [   1,  484,   17, ...,    2,    0,    0],
        [   1,  484,   17, ...,    2,    0,    0]], dtype=int32))

In [15]:
# Вычисляем максимальную длину тензоров
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
max_length_targ, max_length_inp

(10, 20)

In [16]:
# Создаем тренировочные и валидационные датасеты
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# размеры датасетов
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [17]:
input_tensor_train, target_tensor_train

(array([[   1,   28, 1947, ...,    0,    0,    0],
        [   1,  614,    9, ...,    0,    0,    0],
        [   1,   33,   75, ...,    0,    0,    0],
        ...,
        [   1,   20,   27, ...,    0,    0,    0],
        [   1,   16,  145, ...,    0,    0,    0],
        [   1,    6,  137, ...,    0,    0,    0]], dtype=int32),
 array([[   1,   21, 1166, ...,    0,    0,    0],
        [   1,   30,    6, ...,    0,    0,    0],
        [   1,   15,    6, ...,    2,    0,    0],
        ...,
        [   1,    6,   19, ...,    0,    0,    0],
        [   1,   20,  123, ...,    0,    0,    0],
        [   1,    6,  135, ...,    0,    0,    0]], dtype=int32))

In [18]:
#функция получения из токена текста (выводим токен и его индекс)
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [19]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
28 ----> questo
1947 ----> capita
34 ----> molto
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
21 ----> this
1166 ----> happens
8 ----> a
155 ----> lot
3 ----> .
2 ----> <end>


In [20]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
#количество эпох
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
#из каждого элемента (input_tensor_train, target_tensor_train) создает тензор
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
#разбиваем датасет на батчи (списки по 64), удаляя последний неполный батч
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 20]), TensorShape([64, 10]))

In [22]:
from keras.layers import GRU, Concatenate

In [23]:
example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape, example_target_batch.shape)
example_input_batch[0], example_target_batch[0]

(64, 20) (64, 10)


(<tf.Tensor: shape=(20,), dtype=int32, numpy=
 array([   1,   61,   49, 5276,    5,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(10,), dtype=int32, numpy=
 array([   1,   59,   19,    6,   50, 1744,    7,    2,    0,    0],
       dtype=int32)>)

In [24]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.Bidirectional(GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform'))


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state_frw, state_bcw = self.gru(x, initial_state = hidden)
    return state_frw, state_bcw

  # def initialize_hidden_state(self):
  #   return tf.zeros((self.batch_sz, self.enc_units))

  def initialize_hidden_state(self):
      return [tf.zeros((self.batch_sz, self.enc_units)) for i in range(2)]

In [25]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# sample input
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_hidden = encoder.initialize_hidden_state()
forward_hidden, backward_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder forward_h shape: (batch size, units) {}'.format(forward_hidden.shape))
print('Encoder backward_h shape: (batch size, units) {}'.format(backward_hidden.shape))


Encoder forward_h shape: (batch size, units) (64, 1024)
Encoder backward_h shape: (batch size, units) (64, 1024)


In [26]:
sample_hidden

[<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]

In [27]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(2*self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, encoder_fwd_state, encoder_back_state):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=Concatenate(axis=-1)([encoder_fwd_state, encoder_back_state]))

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [28]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      forward_hidden, backward_hidden)

In [29]:
decoder_sample_x.shape

TensorShape([64, 6368])

In [30]:
decoder_sample_h.shape

TensorShape([64, 2048])

## Define the optimizer and the loss function

In [31]:
#оптимизатор
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

#функция потерь
def loss_function(real, pred):
      #делаем инверсию значений сравнения каждого из real с нулем (возвращается true или false)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
      #применяем функцию ошибок к реальным данным и предсказанным
    loss_ = loss_object(real, pred)
      #приводим тензор mask к новому типу loss_.dtype
    mask = tf.cast(mask, dtype=loss_.dtype)
      #умножаем loss_ на mask
    loss_ *= mask
      # возвращаем среднее значениe всех элементов
    return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [32]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [33]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    forward_hidden, backward_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]): # Максимальная длина последовательности
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, forward_hidden, backward_hidden)

      # считаем лосс по каждому предсказанному слову
      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [34]:
EPOCHS = 15

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.8308
Epoch 1 Batch 100 Loss 2.0919
Epoch 1 Batch 200 Loss 1.8328
Epoch 1 Batch 300 Loss 1.7458
Epoch 1 Batch 400 Loss 1.5433
Epoch 1 Batch 500 Loss 1.4056
Epoch 1 Batch 600 Loss 1.2393
Epoch 1 Batch 700 Loss 1.0977
Epoch 1 Batch 800 Loss 1.0489
Epoch 1 Batch 900 Loss 0.9637
Epoch 1 Batch 1000 Loss 0.7245
Epoch 1 Batch 1100 Loss 0.7699
Epoch 1 Batch 1200 Loss 0.7833
Epoch 1 Loss 1.3075
Time taken for 1 epoch 136.09841394424438 sec

Epoch 2 Batch 0 Loss 0.5226
Epoch 2 Batch 100 Loss 0.5093
Epoch 2 Batch 200 Loss 0.3965
Epoch 2 Batch 300 Loss 0.4098
Epoch 2 Batch 400 Loss 0.4137
Epoch 2 Batch 500 Loss 0.5267
Epoch 2 Batch 600 Loss 0.4945
Epoch 2 Batch 700 Loss 0.4352
Epoch 2 Batch 800 Loss 0.4103
Epoch 2 Batch 900 Loss 0.3109
Epoch 2 Batch 1000 Loss 0.3932
Epoch 2 Batch 1100 Loss 0.2886
Epoch 2 Batch 1200 Loss 0.2579
Epoch 2 Loss 0.4061
Time taken for 1 epoch 121.76422643661499 sec

Epoch 3 Batch 0 Loss 0.1636
Epoch 3 Batch 100 Loss 0.1738
Epoch 3 Batch 200 Loss 0.2

In [35]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units)) for i in range(2)]
  enc_hidden = encoder(inputs, hidden)

  forward_hidden, backward_hidden = enc_hidden

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, forward_hidden, backward_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [36]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [37]:
# загружаем последний checkpoint in checkpoint_dir
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [38]:
translate('il mio secchio')

Input: <start> il mio secchio <end>
Predicted translation: it's my seat . <end> 


In [39]:
translate('è bello')

Input: <start> bello <end>
Predicted translation: that's pretty . <end> 


In [40]:
translate('il tempo e soleggiato')

Input: <start> il tempo e soleggiato <end>
Predicted translation: time is on . <end> 


In [41]:
translate('ci incontreremo oggi')

Input: <start> ci incontreremo oggi <end>
Predicted translation: i'll meet him today . <end> 


модель не достаточно точная, но это лучшее, что пока получилось